# Antibiofilm Peptides Prediction

#Import modules

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import numpy as np
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFECV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn import metrics

## Data preprocessing

###### Load peptide datasets

In [48]:
neg_df=pd.read_csv(#Enter your file path to non antibiofilm peptide) 
pos_df=pd.read_csv(#Enter your file path to antibiofilm peptides) 
inde_df=pd.read_csv(#Enter your file path to independent data) 

### Feature Dataset

###### NMR based features:

In [10]:
nmr_df=pd.read_csv(#Enter your file path)
nmr_df = (nmr_df.set_index('peptide')).drop(index=inde_df['Sequence'].tolist())
nmr_df = nmr_df.drop_duplicates()
nmr_df.head()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F31,F32,F33,F34,F35,F36,F37,F38,F39,F40
peptide,,,,,,,,,,,,,,,,,,,,,
AKDEH,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.25,0.25,...,20.0,20.0,20.0,40.0,60.0,0.0,0.0,0.2,0.2,0.6
AKTVQ,0.0,0.0,0.0,0.50,0.0,0.0,0.0,0.00,0.25,0.00,...,20.0,20.0,20.0,60.0,80.0,0.2,0.0,0.2,0.0,0.6
ARNQT,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.25,0.00,0.25,...,40.0,40.0,40.0,40.0,60.0,0.2,0.0,0.2,0.2,0.4
DRVGA,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.25,0.50,...,0.0,0.0,0.0,40.0,60.0,0.0,0.0,0.2,0.2,0.6
EKMIG,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.50,...,0.0,0.0,20.0,60.0,80.0,0.0,0.0,0.0,0.2,0.8


###### Physicochemical Descriptors

In [11]:
phy_df=pd.read_csv(#Enter your file path)
phy_df = (phy_df.set_index('peptide')).drop(index=inde_df['Sequence'].tolist())
phy_df = phy_df.drop_duplicates()
phy_df.head()

,Length,NumberOfA,NumberOfR,NumberOfN,NumberOfD,NumberOfC,NumberOfQ,NumberOfE,NumberOfG,NumberOfH,...,Charged,Basic,Acidic,PI,ChargeInPH8,Carbon,Hydrogen,Nitrogen,Oxygen,Sulfur
peptide,,,,,,,,,,,,,,,,,,,,,
AKDEH,5.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,4.0,2.0,2.0,5.23,-1.37865,24.0,38.0,8.0,10.0,0.0
AKTVQ,5.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,0.0,9.38,0.61031,23.0,43.0,7.0,8.0,0.0
ARNQT,5.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,0.0,10.35,0.61315,22.0,40.0,13.0,9.0,0.0
DRVGA,5.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,2.0,1.0,1.0,6.15,-0.38677,20.0,36.0,11.0,8.0,0.0
EKMIG,5.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,2.0,1.0,1.0,6.21,-0.38958,24.0,44.0,6.0,8.0,1.0


###### Amino Acid Composition (AAC) Features

In [12]:
aac_df=pd.read_csv(#Enter your file path)
aac_df = (aac_df.set_index('seq')).drop(index=inde_df['Sequence'].tolist())
aac_df = aac_df.drop_duplicates()
aac_df.head()

,A,C,E,D,G,F,I,H,K,M,...,N,Q,P,S,R,T,W,V,Y,Unnamed: 21
seq,,,,,,,,,,,,,,,,,,,,,
AKDEH,16.667,0.0,16.667,16.667,0.000,0.0,0.000,16.667,16.667,0.000,...,0.000,0.000,0.0,0.0,0.000,0.000,0.0,0.000,0.0,NaN
AKTVQ,16.667,0.0,0.000,0.000,0.000,0.0,0.000,0.000,16.667,0.000,...,0.000,16.667,0.0,0.0,0.000,16.667,0.0,16.667,0.0,NaN
ARNQT,16.667,0.0,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000,...,16.667,16.667,0.0,0.0,16.667,16.667,0.0,0.000,0.0,NaN
DRVGA,16.667,0.0,0.000,16.667,16.667,0.0,0.000,0.000,0.000,0.000,...,0.000,0.000,0.0,0.0,16.667,0.000,0.0,16.667,0.0,NaN
EKMIG,0.000,0.0,16.667,0.000,16.667,0.0,16.667,0.000,16.667,16.667,...,0.000,0.000,0.0,0.0,0.000,0.000,0.0,0.000,0.0,NaN


###### Composition, Transition and distribution (CTD) Features

In [13]:
ctd_df=pd.read_csv(#Enter your file path)
ctd_df = (ctd_df.set_index('seq')).drop(index=inde_df['Sequence'].tolist())
ctd_df = ctd_df.drop_duplicates()
ctd_df.head()

,NormalizedVDWVD1075,MolecularWeightD3050,SolubilityInWaterC2,SolubilityInWaterC3,SolubilityInWaterC1,NormalizedVDWVC1,NormalizedVDWVC3,NormalizedVDWVC2,SolventAccessibilityD1025,PolarityD1100,...,PPIHotspotPropBoganC2,PPIHotspotPropBoganC3,PDNAIPropAhmadD2075,PLVBSKhazanovD2075,PPIHotspotPropBoganD1025,NoHydroBondAccSideChainD3050,PolarityC1,PolarityC3,PolarityC2,Unnamed: 505
seq,,,,,,,,,,,,,,,,,,,,,
AKDEH,16.667,0.000,0.333,0.167,0.333,0.333,0.333,0.167,16.667,0.000,...,0.333,0.000,50.000,50.000,83.333,16.667,0.000,0.333,0.167,NaN
AKTVQ,16.667,0.000,0.333,0.000,0.500,0.333,0.167,0.333,66.667,66.667,...,0.500,0.167,16.667,50.000,33.333,16.667,0.167,0.167,0.167,NaN
ARNQT,16.667,33.333,0.333,0.000,0.500,0.333,0.167,0.333,16.667,0.000,...,0.500,0.000,16.667,50.000,50.000,16.667,0.000,0.167,0.333,NaN
DRVGA,66.667,33.333,0.167,0.167,0.500,0.500,0.167,0.167,83.333,50.000,...,0.333,0.167,50.000,50.000,33.333,50.000,0.167,0.167,0.000,NaN
EKMIG,83.333,0.000,0.500,0.000,0.333,0.167,0.333,0.333,83.333,66.667,...,0.500,0.000,16.667,66.667,66.667,50.000,0.333,0.167,0.167,NaN


###### Dipeptide composition (DPC) Features

In [14]:
dpc_df=pd.read_csv(#Enter your file path)
dpc_df = (dpc_df.set_index('seq')).drop(index=inde_df['Sequence'].tolist())
dpc_df = dpc_df.drop_duplicates()
dpc_df.head()

,GW,GV,GT,GS,GR,GQ,GP,GY,GG,GF,...,AQ,AP,AS,AR,AT,AW,AV,AY,VK,Unnamed: 401
seq,,,,,,,,,,,,,,,,,,,,,
AKDEH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
AKTVQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
ARNQT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,NaN
DRVGA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
EKMIG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


###### Concatenating Features and class assignment

In [15]:
features_df=pd.concat([nmr_df,phy_df,aac_df,ctd_df,dpc_df],axis=1)
features_df = features_df.dropna(how='all',axis=1)
features_df = features_df.drop_duplicates()

pos_features_df = features_df.reindex(index = pos_df[1].tolist())
pos_features_df['Class'] = 1

neg_features_df = features_df.reindex(index = neg_df[1].tolist())
neg_features_df['Class'] = 0

features_df = pd.concat([pos_features_df, neg_features_df])
features_df = features_df.dropna()
features_df.head()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,AQ,AP,AS,AR,AT,AW,AV,AY,VK,Class
GWGSFFKKAAHVGKHVGKAALTHYL,0.0,0.00000,0.04348,0.04348,0.0,0.0,0.0,0.08696,0.17391,0.21739,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,1
LLGDFFRKSKEKIGKEFKRIVQRIKDFLRNLVPRTES,0.0,0.00000,0.02778,0.02778,0.0,0.0,0.0,0.02778,0.11111,0.33333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,1
GRFKRFRKKFKKLFKKLSPVIPLLHL,0.0,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.00000,0.16000,0.40000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,1
GGLRSLGRKILRAWKKYGPIIVPIIRI,0.0,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.08000,0.24000,0.32000,...,0.0,0.0,0.0,0.0,0.0,3.7,0.0,0.0,0.00,1
RGLRRLGRKIAHGVKKYGPTVLRIIRIA,0.0,0.03704,0.00000,0.03704,0.0,0.0,0.0,0.03704,0.11111,0.44444,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.57,1


In [16]:
all_features = features_df.columns[:-1]

###### Removing Constant features

In [17]:
constant_filter = VarianceThreshold(threshold=0)
constant_filter.fit(features_df[all_features].values)
constant_columns = [f for f in all_features if f not in all_features[constant_filter.get_support()]]

print("Constant Features:",len(constant_columns))
print("Remaining Features:",len(all_features[constant_filter.get_support()]))

# Removing constant features
all_features = all_features[constant_filter.get_support()]
features_df = features_df[list(all_features)+['Class']]

Constant Features: 56
Remaining Features: 944


###### Removing Quasi-Constant features

In [18]:
qconstant_filter = VarianceThreshold(threshold=0.01)
qconstant_filter.fit(features_df[all_features].values)
qconstant_columns = [f for f in all_features if f not in all_features[qconstant_filter.get_support()]]

print("Constant Features:",len(qconstant_columns))
print("Remaining Features:",len(all_features[qconstant_filter.get_support()]))

# Removing qconstant features
all_features = all_features[qconstant_filter.get_support()]
features_df = features_df[list(all_features)+['Class']]

Constant Features: 24
Remaining Features: 920


###### Removing Duplicate Features

In [19]:
unique_features = features_df.T.drop_duplicates(keep='first').T
duplicated_features = [dup_col for dup_col in features_df.columns if dup_col not in unique_features.columns]

###### Identification of highly correlated features

In [20]:
correlation_matrix = unique_features.corr()

In [ ]:
threshold = 0.95
predictors = unique_features.drop(['Class'], axis = 1) 
criterion = unique_features["Class"]

def high_cor_function(df):
    cor = df.corr()
    corrm = np.corrcoef(df.transpose())
    corr = corrm - np.diagflat(corrm.diagonal())
    print("max corr:",corr.max(), ", min corr: ", corr.min())
    c1 = cor.stack().sort_values(ascending=False).drop_duplicates()
    high_cor = c1[c1.values!=1]    
    thresh = threshold 
    display(high_cor[high_cor>thresh])

high_cor_function(predictors)

In [22]:
threshold = 0.95
correlated_features = set()

for i in range(len(correlation_matrix .columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > threshold:
            colname = correlation_matrix.columns[i]
            correlated_features.add(colname)

In [23]:
unique_features = unique_features.drop(correlated_features, axis = 1) 

###### Eliminating Features using RFECV

In [25]:
all_features = unique_features.columns[:-1]

scaler = MinMaxScaler()
unique_features_norm = unique_features.copy()
unique_features_norm[all_features] = scaler.fit_transform(unique_features[all_features])
X = unique_features_norm[all_features]
y = unique_features_norm['Class']

# X = unique_features[all_features]
# y = unique_features['Class']
trainX, testX, trainY, testY = train_test_split(X, y, test_size = 0.2)

In [ ]:
rfe = RFECV(estimator=RandomForestClassifier(), step=1, cv=10)
rfe.fit(trainX, trainY)

In [ ]:
selected_rfe_features = pd.DataFrame({'Feature':list(unique_features_norm.columns[:-1]),
                                      'Ranking':rfe.ranking_})
selected_rfe_features.sort_values(by='Ranking')

##### Train and test Data

In [29]:
trainX = trainX[list(selected_rfe_features.loc[selected_rfe_features["Ranking"]==1]['Feature'])]
testX = testX[list(selected_rfe_features.loc[selected_rfe_features["Ranking"]==1]['Feature'])]

## Predictions

##### Support Vector Machine

In [30]:
svm_clf = SVC(gamma='auto')
svm_scores = cross_val_score(svm_clf, trainX, trainY, cv=10, n_jobs=-1)
svm_scores.mean()

0.7542105263157894

In [53]:
svm_clf.fit(trainX, trainY)
svm_pred = svm_clf.predict(testX)

print("Accuracy:", metrics.accuracy_score(testY, svm_pred))
print("Precision:", metrics.precision_score(testY, svm_pred))
print("Recall:", metrics.recall_score(testY, svm_pred))

Accuracy: 0.7551020408163265
Precision: 0.7333333333333333
Recall: 1.0


### RandomForestClassifier

In [61]:
forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)
forest_scores = cross_val_score(forest_clf, trainX, trainY, cv=10, n_jobs=-1)
forest_scores.mean()


0.923157894736842

In [62]:
forest_clf.fit(trainX, trainY)
forest_pred = forest_clf.predict(testX)
print("Accuracy:",metrics.accuracy_score(testY, forest_pred))
print("Precision:",metrics.precision_score(testY, forest_pred))
print("Recall:",metrics.recall_score(testY, forest_pred))

Accuracy: 0.9387755102040817
Precision: 0.9166666666666666
Recall: 1.0
